In [89]:
import pandas as pd
import numpy as np
from math import floor

In [90]:
df = pd.read_csv('new_accuracy_table.csv')

In [91]:
df.dropna(subset=['gated_count'], inplace=True)

In [92]:
df.at[:,'gated_count'] = np.array((np.vectorize(int))(df['gated_count']), dtype='int')

In [93]:
def fix_input(num: float) -> str:
    if pd.isna(num):
        return "-"
    n = floor(num)
    return f"{n:02d}"

fix_input_column = np.vectorize(fix_input)

In [94]:
df.at[:,'input'] = fix_input_column(df['input'])

In [95]:
df.at[:, "gate"] = np.where(pd.isna(df['gate']), df["strain_name"], df["gate"])

In [96]:
df.head()

,lab_id,strain_name,plate_id,well,count,count_live,experiment_id,filename,gate,gated_count,...,std_correct_low_threshold,std_correct_low_threshold_live,std_correct_threshold,std_correct_threshold_live,std_log_gfp,std_log_gfp_gated,std_log_gfp_gated_live,std_log_gfp_live,threshold,threshold_live
0,r1c5va879uaex_r1c639xp952g4,NOR-00-Control,0,C12,NaN,NaN,NaN,NaN,NOR-00-Control,25743,...,NaN,NaN,NaN,NaN,NaN,0.286968,NaN,NaN,NaN,NaN
1,r1c5va879uaex_r1c639xp952g4,WT-Live-Control,0,A12,NaN,NaN,NaN,NaN,WT-Live-Control,16591,...,NaN,NaN,NaN,NaN,NaN,0.469159,NaN,NaN,NaN,NaN
2,r1c5va879uaex_r1c639xp952g4,https://hub.sd2e.org/user/sd2e/design/UWBF_169...,0,B03,28297.0,24410.0,NaN,/work/05202/dbryce/xplan-reactor/fcs/YeastGate...,XOR,24715,...,0.002825,0.002889,0.002825,0.002889,0.981708,0.395890,0.395018,0.934420,5.257244,5.284602
3,r1c5va879uaex_r1c639xp952g4,https://hub.sd2e.org/user/sd2e/design/UWBF_169...,0,D03,27568.0,23178.0,NaN,/work/05202/dbryce/xplan-reactor/fcs/YeastGate...,XOR,23737,...,0.002714,0.002791,0.002714,0.002791,1.035838,0.422622,0.421670,0.980817,5.257244,5.284602
4,r1c5va879uaex_r1c639xp952g4,https://hub.sd2e.org/user/sd2e/design/UWBF_169...,0,A10,29862.0,15228.0,NaN,/work/05202/dbryce/xplan-reactor/fcs/YeastGate...,XOR,25483,...,0.002156,0.003665,0.002156,0.003665,0.872558,0.327265,0.327281,0.928051,5.257244,5.284602


In [97]:
df.input.unique()

array(['-', '11', '10', '01', '00'], dtype=object)

In [98]:
df.columns

Index(['lab_id', 'strain_name', 'plate_id', 'well', 'count', 'count_live',
       'experiment_id', 'filename', 'gate', 'gated_count', 'gated_live_count',
       'growth_media_1', 'growth_media_2', 'id', 'inc_temp', 'inc_time_1',
       'inc_time_2', 'index', 'input', 'lab', 'mean_correct_classifier',
       'mean_correct_classifier_live', 'mean_correct_high_classifier',
       'mean_correct_high_classifier_live', 'mean_correct_high_threshold',
       'mean_correct_high_threshold_live', 'mean_correct_low_classifier',
       'mean_correct_low_classifier_live', 'mean_correct_low_threshold',
       'mean_correct_low_threshold_live', 'mean_correct_threshold',
       'mean_correct_threshold_live', 'mean_log_gfp', 'mean_log_gfp_gated',
       'mean_log_gfp_gated_live', 'mean_log_gfp_live', 'media', 'od',
       'od_cutoff', 'output', 'plan', 'replicate', 'source_container',
       'std_correct_classifier', 'std_correct_classifier_live',
       'std_correct_high_classifier', 'std_correct_high_

In [99]:
df = df[['lab_id', 'well', 'count', 'gate', 'input', 'output', 'gated_count', 'growth_media_1', 'inc_temp']]

In [100]:
df.shape[0]

7351

In [101]:
df[df['gated_count'] >= 10_000].shape[0]

7351

In [102]:
print("{:,d}".format(df[df['gated_count'] >= 10_000]['gated_count'].sum()))

145,966,885


In [122]:
all_counts = \
    df[df['gated_count'] >= 10_000][['gate', 'input', 'gated_count']]\
         .groupby(['gate', 'input'])\
         .agg(['sum', 'count'])\
         .rename(columns={'gated_count': 'Gated count', 'sum': 'events', 'count': 'replicates'})

In [137]:
all_counts.loc[:, ('Gated count', 'events')] = all_counts.loc[:, ('Gated count', 'events')].astype(int).apply(lambda x: "{:,}".format(x))

In [123]:
all_counts.reset_index(drop=False)['gate'].unique()

array(['AND', 'NAND', 'NOR', 'NOR-00-Control', 'OR', 'WT-Live-Control',
       'XNOR', 'XOR'], dtype=object)

In [138]:
all_counts.query("gate not in ['NOR-00-Control', 'WT-Live-Control']")

Gated count           
                events replicates
gate input                       
AND  00      4,259,281        257
     01      4,459,111        246
     10      4,044,177        234
     11      4,791,203        222
NAND 00      7,402,180        345
     01      7,412,703        352
     10      6,778,828        316
     11      7,308,418        337
NOR  00      8,573,805        368
     01      5,913,758        317
     10      6,133,394        341
     11      7,515,656        342
OR   00      4,343,223        249
     01      4,928,624        237
     10      3,954,589        249
     11      5,170,802        242
XNOR 00      7,111,913        312
     01      4,818,814        294
     10      5,551,739        290
     11      6,436,798        293
XOR  00      6,286,085        353
     01      5,287,884        318
     10      7,227,426        329
     11      7,142,248        346

In [150]:
np.vectorize(lambda x: int(x.replace(",","")))(all_counts.query("gate not in ['NOR-00-Control', 'WT-Live-Control']")[('Gated count', 'events')]).sum()

142852659

In [151]:
all_counts.query("gate not in ['NOR-00-Control', 'WT-Live-Control']")[('Gated count', 'replicates')].sum()

7189

In [139]:
all_counts.query("gate not in ['NOR-00-Control', 'WT-Live-Control']").to_latex('gate_counts_cp1.tex')

In [140]:
all_counts.query("gate in ['NOR-00-Control', 'WT-Live-Control']").to_latex('control_counts_cp1.tex')

In [152]:
np.vectorize(lambda x: int(x.replace(",","")))(all_counts.query("gate in ['NOR-00-Control', 'WT-Live-Control']")[('Gated count', 'events')]).sum()

3114226